In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data procenotebookb022d1b41fssing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sarcasm/train-balanced-sarc.csv.gz
/kaggle/input/sarcasm/train-balanced-sarcasm.csv
/kaggle/input/sarcasm/test-balanced.csv
/kaggle/input/sarcasm/test-unbalanced.csv


In [3]:
!pip install setfit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 10.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=eea3eeec0ed994355f39aa1bbb499da4d91e32ef9f27bc694681dbe4d25a4265
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0


In [4]:
# Data processing
import pandas as pd
import numpy as np

# Modeling
import tensorflow as tf

# Hugging Face Dataset
from datasets import Dataset

# Model performance evaluation
# import evaluate

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
import csv

# Define the CSV file path
csv_file = '/kaggle/input/sarcasm/train-balanced-sarcasm.csv'

# Initialize an empty list to store the data
data = []

# Open the CSV file
with open(csv_file, mode='r', newline='') as file:
    # Create a CSV reader
    reader = csv.reader(file)

    # Skip the header row if it exists (optional)
    header = next(reader)

    # Iterate over the rows in the CSV file
    for row in reader:
        # Check if the value of the "subreddit" column is "politics"
        if row[3] == 'politics':
            # If it is, add the first two columns to the data list
            data.append(row[:2])

In [6]:
len(data)

39496

In [7]:
data = pd.DataFrame(data)
# Assuming 'data_test' is your DataFrame
data = data.rename(columns={0: 'label', 1: 'headline'})

In [8]:
data [0:3]

,label,headline
0,0,NC and NH.
1,0,I think a significant amount would be against ...
2,0,because it's what really bothers him... and it...


In [9]:
# Convert the DataFrame to a list of dictionaries
data_list = data.to_dict(orient='records')

# Rename the 'headline' key to 'sentence' to match your desired format
for item in data_list:
    item['sentence'] = item.pop('headline')

In [10]:
# Convert 'label' to integers and add 'idx' to each dictionary
for idx, data in enumerate(data_list):
    data['label'] = int(data['label'])
    data['idx'] = idx

In [11]:
from sklearn.model_selection import train_test_split

# Split the data into training (80%) and testing (20%)
train_data, test_data = train_test_split(data_list, test_size=0.2, random_state=42)


In [12]:
len(train_data)

31596

In [13]:
len(test_data)

7900

In [14]:
from datasets import Dataset, DatasetDict

In [15]:
dataset_dict = DatasetDict({
    "train": Dataset.from_dict({key: [item[key] for item in train_data] for key in train_data[0]}),
    "test" : Dataset.from_dict({key: [item[key] for item in test_data] for key in test_data[0]})
})

In [16]:
dataset_dict["train"][0:10]

{'label': [0, 0, 0, 0, 0, 1, 1, 0, 1, 0],
 'sentence': ['What people are we talking about?',
  'He did, by leveraging his endorsement for putting progressive policies into the Democratic party platform.',
  "I mean, i'd be more than happy to receive a check in the mail, if you're listening Hillary...",
  'The contradiction is real..',
  'Meanwhile in Loompaland, Trump is winning and anyone that says otherwise is a shill.',
  'The US is just an inherently more violent place, just look at our track record.',
  'LIBRULS ARE WHY CONSERVATIVES KILL PEOPLE!',
  'Those are rigged too, *duh*',
  'They get abortions all the time.',
  'That sounds like enough'],
 'idx': [38146, 3353, 15392, 7492, 3202, 29470, 38016, 1149, 35816, 28515]}

In [17]:
from setfit import sample_dataset

train_dataset = dataset_dict["train"]
train_dataset[0:10]

{'label': [0, 0, 0, 0, 0, 1, 1, 0, 1, 0],
 'sentence': ['What people are we talking about?',
  'He did, by leveraging his endorsement for putting progressive policies into the Democratic party platform.',
  "I mean, i'd be more than happy to receive a check in the mail, if you're listening Hillary...",
  'The contradiction is real..',
  'Meanwhile in Loompaland, Trump is winning and anyone that says otherwise is a shill.',
  'The US is just an inherently more violent place, just look at our track record.',
  'LIBRULS ARE WHY CONSERVATIVES KILL PEOPLE!',
  'Those are rigged too, *duh*',
  'They get abortions all the time.',
  'That sounds like enough'],
 'idx': [38146, 3353, 15392, 7492, 3202, 29470, 38016, 1149, 35816, 28515]}

In [18]:
eval_dataset = dataset_dict["test"]

In [19]:
model_id = "sentence-transformers/paraphrase-mpnet-base-v2"

In [20]:
from setfit import SetFitModel

num_classes = len(train_dataset.unique("label"))
# Assuming train_dataset is a list of dictionaries and 'label' is the key for the label
# unique_labels = set(data['label'] for data in train_dataset)

# # Calculate the number of unique classes
# num_classes = len(unique_labels)
model = SetFitModel.from_pretrained(model_id, use_differentiable_head=True, head_params={"out_features": num_classes})

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [21]:
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitTrainer

trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss_class=CosineSimilarityLoss,
    num_iterations=20,
    column_mapping={"sentence": "text", "label": "label"},
)

In [22]:
trainer.freeze()
trainer.train(body_learning_rate=1e-5, num_epochs=1)

Applying column mapping to training dataset


Generating Training Pairs:   0%|          | 0/20 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 1263840
  Num epochs = 1
  Total optimization steps = 78990
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/78990 [00:00<?, ?it/s]

In [24]:
trainer.unfreeze(keep_body_frozen=True)
trainer.train(learning_rate=1e-2, num_epochs=25)

Applying column mapping to training dataset
The `max_length` is `None`. Using the maximum acceptable length according to the current model body: 512.


Epoch:   0%|          | 0/25 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [25]:
trainer.evaluate()


Applying column mapping to evaluation dataset
***** Running evaluation *****


Batches:   0%|          | 0/247 [00:00<?, ?it/s]

{'accuracy': 0.7889873417721519}

In [26]:
model._save_pretrained('save_directory')